In [5]:
from torchvision.datasets import MNIST
from torchvision import transforms

mean, std = 0.1307, 0.3081

train_dataset = MNIST('../t-sne/data/MNIST', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((mean,), (std,))
                             ]))
test_dataset = MNIST('../t-sne/data/MNIST', train=False, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((mean,), (std,))
                            ]))
n_classes = 10

In [6]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
from sklearn import decomposition
from trainer import fit
import numpy as np
#cuda = torch.cuda.is_available()
cuda = False
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

mnist_classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']

def plot_embeddings(embeddings, targets, xlim=None, ylim=None):
    plt.figure(figsize=(10,10))
    for i in range(10):
        inds = np.where(targets==i)[0]
        plt.scatter(embeddings[inds,0], embeddings[inds,1], alpha=0.5, color=colors[i])
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    plt.legend(mnist_classes)

def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader.dataset), 256))
        labels = np.zeros(len(dataloader.dataset))
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels[k:k+len(images)] = target.numpy()
            k += len(images)
    return embeddings, labels

def extract_raw(dataloader):
    rawdata = np.zeros((1, 784))
    #k = 0
    for images, target in dataloader:
        for image in images:
            rawdata=np.insert(rawdata,rawdata.shape[0],image.numpy().flatten(),axis=0)
        #rawdata[k:k+len(images)] = torch.flatten(images, start_dim=1)
        #k += len(images)
    rawdata=np.delete(rawdata, 0, 0)
    print(rawdata.shape)
    return rawdata

def extract_pca(rawdata):
    pca = decomposition.PCA(n_components=10)
    pca.fit(rawdata)
    pca_result = pca.transform(rawdata)
    return pca_result

In [7]:
from datasets import TripletMNIST

triplet_train_dataset = TripletMNIST(train_dataset) # Returns triplets of images
triplet_test_dataset = TripletMNIST(test_dataset)
batch_size = 1024
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters
from networks import EmbeddingNet, TripletNet
from losses import TripletLoss

margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 2000
log_interval = 100

In [ ]:
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

Train: [0/60000 (0%)]	Loss: 0.695995
Epoch: 1/2000. Train set: Average loss: 0.0893
Epoch: 1/2000. Validation set: Average loss: 0.0360
Train: [0/60000 (0%)]	Loss: 0.024018
Epoch: 2/2000. Train set: Average loss: 0.0254
Epoch: 2/2000. Validation set: Average loss: 0.0187
Train: [0/60000 (0%)]	Loss: 0.031609


In [12]:
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)
#train_embeddings_tl, train_labels_tl = extract_embeddings(train_loader, model)
#plot_embeddings(train_embeddings_tl, train_labels_tl)
val_embeddings_tl, val_labels_tl = extract_embeddings(test_loader, model)
#plot_embeddings(val_embeddings_tl, val_labels_tl)
torch.save(val_embeddings_tl,'triplet_embedding.pth')

In [13]:
torch.save(val_labels_tl,'label.pth')

In [14]:
rawdata = extract_raw(test_loader)

(10000, 784)


In [15]:
torch.save(rawdata,'rawdata.pth')

In [16]:
pca_result = extract_pca(rawdata)

In [17]:
torch.save(pca_result,'pca_result.pth')